In [249]:
import json
dep = [('Closure', 63), ('Closure', 93), ('Time', 21), ('Lang', 2)]
dep.extend([('Cli', 6)])
dep.extend([('Collections', i) for i in range(1, 25)])
print(len(dep))
def get_one_funcs(path):
    with open(path) as f:
        infos_one_func = []
        for line in f:
            info = json.loads(line)
            if 'extract_src_erro' in info or 'extract_test_erro' in info:
                continue
            if (info['project'], info['bug_id']) in dep:
                continue
            if len(info['erro_repairs'])!=1:
                continue
            erro_repair = info['erro_repairs'][0]
            if len(erro_repair['if_one_function']) != 1:
                continue
            elif not all(erro_repair['if_one_function']):
                continue
            infos_one_func.append(info)
    return infos_one_func

29


In [3]:
import re
r = 'Failing tests: 1\n  - org.apache.commons.'
re.findall("Failing tests: (\d)\n", r)

['1']

In [250]:
infos_one_func_1 = get_one_funcs("/home/zhoushiqi/workplace/apr/data/df4_process_data/all_info_defects4j-1.2.0_external.jsonl")
# infos_one_func_2 = get_one_funcs("/home/zhoushiqi/workplace/apr/data/df4_process_data/all_info_defects4j-2.0.1.jsonl")

In [136]:
len(infos_one_func_1)

255

In [455]:
infos_one_func_1[0]['erro_repairs'][0]['src_code'][0]

'    public void atan2(final double[] y, final int yOffset,\n                      final double[] x, final int xOffset,\n                      final double[] result, final int resultOffset) {\n\n        // compute r = sqrt(x^2+y^2)\n        double[] tmp1 = new double[getSize()];\n        multiply(x, xOffset, x, xOffset, tmp1, 0);      // x^2\n        double[] tmp2 = new double[getSize()];\n        multiply(y, yOffset, y, yOffset, tmp2, 0);      // y^2\n        add(tmp1, 0, tmp2, 0, tmp2, 0);                 // x^2 + y^2\n        rootN(tmp2, 0, 2, tmp1, 0);                     // r = sqrt(x^2 + y^2)\n\n        if (x[xOffset] >= 0) {\n\n            // compute atan2(y, x) = 2 atan(y / (r + x))\n            add(tmp1, 0, x, xOffset, tmp2, 0);          // r + x\n            divide(y, yOffset, tmp2, 0, tmp1, 0);       // y /(r + x)\n            atan(tmp1, 0, tmp2, 0);                     // atan(y / (r + x))\n            for (int i = 0; i < tmp2.length; ++i) {\n                result[resultOf

In [457]:
ids_1 = {(info['project'], info['bug_id']) for info in infos_one_func_1}
ids_2 = {(info['project'], info['bug_id']) for info in infos_one_func_2}
new_ids_2 = set(ids_2) - set(ids_1)
len(new_ids_2)

231

In [251]:
import utils
utils.write_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/1.2_external.jsonl",infos_one_func_1)

In [28]:
new_ids_2 = set(ids_2)-set(ids_1)

In [29]:
len(new_ids_2)

233

In [7]:
infos_one_func_2_new = []
for info in infos_one_func_2:
    if (info['project'], info['bug_id']) in new_ids_2:
        infos_one_func_2_new.append(info)

In [8]:
len(infos_one_func_2_new)

231

In [12]:
import utils
datas = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/2.0.jsonl")
len(datas)

231

In [11]:
utils.write_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/1.2.jsonl", infos_one_func_1)
utils.write_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/2.0.jsonl", infos_one_func_2_new)

In [32]:
print("  public void testPropertyOfMethod() {\n    testFailure(\"a.protoype.b = {}; \" +\n        \"a.prototype.b.c = function() { this.foo = 3; };\");\n  }")

  public void testPropertyOfMethod() {
    testFailure("a.protoype.b = {}; " +
        "a.prototype.b.c = function() { this.foo = 3; };");
  }


In [4]:
results = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/mixtral/baseline.jsonl")

In [7]:
results[1]

[' The buggy line in the `findMax` method is:\n\n`for (int i = 1; i < numbers.length; i++) {`\n\nIt should be:\n\n`for (int i = 1; i <= numbers.length - 1; i++) {` or simply `for (int i = 1; i < numbers.length; i++) {`\n\n---\n\nThe buggy code in the `parse` method is:\n\n`Number im = parseNumber(source, getRealFormat(), pos);`\n\nIt should be:\n\n`Number im = parseNumber(source, getImaginaryFormat(), pos);`\n\nAlso, in the same method, there is a comparison with the imaginary character that seems to be incorrect:\n\n`source.substring(startIndex, endIndex).compareTo(getImaginaryCharacter()) != 0`\n\nIt should be:\n\n`source.substring(startIndex, endIndex).equals(getImaginaryCharacter())`\n\nBecause the `getImaginaryCharacter()` returns a string with a length of 1, so it should be compared with the equals method instead of compareTo.',
 ' The buggy line in the `findMax` method is `for (int i = 1; i < numbers.length; i++)`. It should be `for (int i = 1; i < numbers.length; i++)` instead.

In [1]:
import utils
import json

In [24]:
import utils
results = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_random_N200_T1_result.jsonl")
# plausible = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_OnlyBuggycode_N100_T1_result.jsonl")
# plausible = [f"{p['project']}_{p['bug_id']}" for p in plausible if "Failing tests: 0\n" in p['result']]
t = 0
# x3 = {}
for result in results:
    # if f"{result['project']}_{result['bug_id']}" in plausible:continue
    # x3[f"{result['project']}_{result['bug_id']}]"] = result['result']
    if "Failing tests: 0\n" in result['result'][:100]:
        t += 1
t

67

In [2]:
def ls2d(ls):
    new = {}
    for l in ls:
        new[f"{l['project']}_{l['bug_id']}"] = l
    return new

In [3]:
import utils
xs = ls2d(utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_random_N1_T1.jsonl"))
ys = ls2d(utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_random_N2_T1.jsonl"))

In [4]:
for x in xs:
    assert xs[x]['prompt'] == ys[x]['prompt']

AssertionError: 

In [9]:
f = open("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process.jsonl")
import json
for x in f:
    y = json.loads(x)
    break

In [12]:
print(y['diff_context'])

@@ -20,5 +20,5 @@

             }
         } while(remaining>0);
 
-        onDead(new TimeoutException("Ping started on "+start+" hasn't completed at "+System.currentTimeMillis()).initCause(e));
+        onDead(new TimeoutException("Ping started on "+start+" hasn't completed at "+System.currentTimeMillis()));//.initCause(e)
     }
